In [ ]:
import os
import matplotlib.pyplot as plt
from database              import DatabaseFromLocalFile
from pipeline              import MasterBias, MasterFlat, BiasCorrectedScienceFrames 
from orderMaskExtraction   import OrderMaskExtraction 
from orderExtraction       import OrderExtraction
from optimalExtraction     import OptimalExtraction
from wavelengthCalibration import WavelengthCalibration

#%matplotlib tk

# Database for pipeline

All the information of a pipeline components result or input will be stored both in a FITS header as well as in a database. This will make searching and filtering information easier. 

Instead of running a database we can use a local ascii file as a database proxy. The following code snipet will generate such a database object. 

For this database object to work properly, it assumes that the local file structure fits what it expects. The observations should be found in the directory: 

`Data/RawData/CalibrationImages/` for calibration images and,
`Data/RawData/ScienceFrames/` for science frames.
 

In [ ]:
db = DatabaseFromLocalFile("pipelineDatabase.txt")
db.save()

The database can now be inspected by opening the file `pipelineDatabase.txt`. 

# Master Bias Image

A master bias image takes as input a list of raw bias images. The input can be given as a path name of the files you want to give as input or their hash number. To find the hash number of a raw image you would need to look into the database.

In [ ]:
raw_bias_paths = ['Data/RawData/CalibrationImages/Bias/20221028T164120_BBBBB_H_0000.fits',
                  'Data/RawData/CalibrationImages/Bias/20221028T164326_BBBBB_H_0000.fits',
                  'Data/RawData/CalibrationImages/Bias/20221028T164524_BBBBB_H_0000.fits',
                  'Data/RawData/CalibrationImages/Bias/20221028T164710_BBBBB_H_0000.fits',
                  'Data/RawData/CalibrationImages/Bias/20221028T164854_BBBBB_H_0000.fits',
                  ]

In [ ]:
master_bias = MasterBias(db, BiasImages=raw_bias_paths)
m_bias_values = master_bias.run("masterBias.fits")

In [ ]:
plt.imshow(m_bias_values)
plt.show()

This rather boring image is obtained by taking the median image of the images that where given as input. As we can see, it is possible to plot this image in python. However there should also be FITS file in the location `Data/ProcessedData/MasterBias` called `masterBias.fits`. 

# Create a Master Flat Image

In [ ]:
raw_flat_paths = ['Data/RawData/CalibrationImages/Flat/20221028T172658_FFFFF_H_0004.fits',
                  'Data/RawData/CalibrationImages/Flat/20221028T173510_FFFFF_H_0004.fits',
                  'Data/RawData/CalibrationImages/Flat/20221028T174304_FFFFF_H_0004.fits',
                  'Data/RawData/CalibrationImages/Flat/20221028T175108_FFFFF_H_0004.fits',
                  'Data/RawData/CalibrationImages/Flat/20221028T175908_FFFFF_H_0004.fits',
                  'Data/RawData/CalibrationImages/Flat/20221028T180713_FFFFF_H_0004.fits']

master_bias_path = "Data/ProcessedData/MasterBias/masterBias.fits"
masterFlat = MasterFlat(db, FlatImages=raw_flat_paths, BiasImages=master_bias_path)
m_Flat_values = masterFlat.run("masterFlat.fits")

In [ ]:
plt.imshow(m_Flat_values)
plt.show()

# Bias corrected Science Image

Before a raw science image gets used in the pipeline we correct for the bias by subtracting a Master Bias image from the science image. This is done in the `BiasCorrectedScienceFrames` component.

In [ ]:
raw_science_paths = ["Data/RawData/ScienceFrames/20221027T175106_TSSSS_H_0900.fits", 
                     "Data/RawData/ScienceFrames/20221027T175724_TSSSS_H_0900.fits"]
master_bias_path = "Data/ProcessedData/MasterBias/masterBias.fits"

for raw_science_path in raw_science_paths:
    calibration = BiasCorrectedScienceFrames(db, ScienceImages=raw_science_path, BiasImages=master_bias_path)
    basename, extension = os.path.splitext(os.path.basename(raw_science_path))
    outputPath = basename + "_BC" + extension
    calibration.run(outputPath)

# Pixel Mask extraction

The components up to know did not give a lot of feedback to the user. This is to be expected since they are doing relativly easy things like taking median of images or substracting images. The components that follow are a bit more complicated. Because of this they have an debug option that allows the user to receive feedback from the component while it is running. This behaviour can be toggled by setting the `debug` option to `1, 2 or 3` with `1` being no feedback and `3` the most feedback. If no value is given, the component gives no feedback by default.

In [ ]:
# Using a master flat field, determine the pixel masks for each order.  

masterflat_path = "Data/ProcessedData/MasterFlat/masterFlat.fits"
maskExtractor = OrderMaskExtraction(db, debug=3, FlatImages=masterflat_path)
x_pixels, y_pixels, flux_values, orders = maskExtractor.run("orderMask.fits")

The compnent returns the extracted x-pixels, y-pixels, flux for every fiber/order. Once we have this mask (because of the x and y-pixels, we can use this mask to extract the pixels from the science images. The flux values will be used later when we use the optimal extraction.

# Science Order Extraction

Now that we have a mask from the flat field, we are able to extract the relavant pixels from the Bias Calibrated Science images. What happens here is pretty straighforward since it simply selects the relevant pixels and saves those together with the mask in a FITS file for every fiber/order. The debug mode here only output an image of the mask overlayed with the science image.

In [ ]:
# Using the pixel mask, extract the orders of a (bias corrected) science image

science_image_paths = ["Data/ProcessedData/BiasCorrectedScience/20221027T175106_TSSSS_H_0900_BC.fits", 
                       "Data/ProcessedData/BiasCorrectedScience/20221027T175724_TSSSS_H_0900_BC.fits"]

order_mask_path    = "Data/ProcessedData/ExtractedOrders/orderMask.fits"
for science_image_path in science_image_paths:
    orderExtractor = OrderExtraction(db, debug=3, ExtractedOrders=order_mask_path, ScienceImages=science_image_path)
    basename, extension = os.path.splitext(os.path.basename(science_image_path))
    outputPath = basename[:-3] + "_EO" + extension
    orderExtractor.run(outputPath)

Now we have extracted the flux from a Master Flat Field and a Bias Corrected Science image we are able to reduce the 2D spectrum to a 1D spectrum. For this we use the Optimal Order extraction alghoritm as described in [Zachmeister et al.](https://www.aanda.org/articles/aa/full_html/2014/01/aa22746-13/aa22746-13.html) 

# Optimal Order Extraction

The optimal order extraction routine is quite a heavy computation. It might take a while to run the component. If we use the maximal debug `3`, the output plots 3 images with slider. The first image is a plot of the average flux of a flat field (average in across the orders) for every fiber/order. The order/fibers can be changed by sliding the slidebar in the image. The second plot is the same, but done for a science image. The last plot is the optimal extracted flux (s_x / f_x). 

In [ ]:
# Extract a 1D spectrum for each order, not yet wavelength calibrated but corrected for
# the instrumental response curve. We call this an "optimal science extraction".

extracted_science_paths = ["Data/ProcessedData/ExtractedOrders/20221027T175106_TSSSS_H_0900_EO.fits", 
                           "Data/ProcessedData/ExtractedOrders/20221027T175724_TSSSS_H_0900_EO.fits"]
extracted_flat_path = "Data/ProcessedData/ExtractedOrders/orderMask.fits"
for extracted_science_path in extracted_science_paths:
    optimalScience = OptimalExtraction(db, debug=3, ExtractedOrders=[extracted_science_path, extracted_flat_path])
    basename, extension = os.path.splitext(os.path.basename(extracted_science_path))
    outputPath = basename[:-3] + "_OP" + extension
    optimalScience.run(outputPath)

# Wavelength Calibration

Finally we are able to run the final (as of this moment) available component of the pipeline: the wavelength calibration. 

In [ ]:
optimal_extracted_science_paths = ["Data/ProcessedData/OptimalExtraction/20221027T175106_TSSSS_H_0900_OP.fits", 
                                   "Data/ProcessedData/OptimalExtraction/20221027T175724_TSSSS_H_0900_OP.fits"]
for optimal_extracted_science_path in optimal_extracted_science_paths:
    wavelength_calibration = WavelengthCalibration(db, debug=1, OptimalExtracted=optimal_extracted_science_path)
    basename, extension = os.path.splitext(os.path.basename(optimal_extracted_science_path))
    outputPath = basename[:-3] + "_WC" + extension
    wavelength_calibration.run(outputPath)

Remark that the database object keeps all the information of newly created images into its memory. However once we close this file all this information gets lost and we are no longer able to access it. We are able to save the information to the text. This file can in turn be used again to create a new database object that contains the same information as the one we currently have. 

In [ ]:
db.save()